### Export Plant Detection Model

In [ ]:
from ultralyticsplus import YOLO
import torch
import onnx

# Load the YOLO model
model = YOLO('foduucom/plant-leaf-detection-and-classification')

# Get the underlying PyTorch model
pytorch_model = model.model  # Extract the actual PyTorch model

# Set the model to evaluation mode
pytorch_model.eval()

# Create a dummy input tensor
dummy_input = torch.randn(1, 3, 640, 640)  # Change dimensions if needed

# Convert to ONNX
onnx_path = "yolov8.onnx"
torch.onnx.export(pytorch_model, dummy_input, onnx_path, opset_version=11)

print("Model successfully converted to ONNX format:", onnx_path)


Model successfully converted to ONNX format: yolov8.onnx


In [ ]:
from ultralyticsplus import YOLO
import torch
import onnx

# Load the YOLO model
model = YOLO('foduucom/plant-leaf-detection-and-classification')



c:\Users\1hasa\miniconda3\envs\old_base\lib\site-packages\ultralytics\nn\tasks.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cp

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
from ultralyticsplus import YOLO
# Access the underlying model and set it to evaluation mode
yolo_model = model.model
yolo_model.eval()  # This is now valid since yolo_model is a PyTorch model

# Example input tensor, for YOLO, it's usually [1, 3, 640, 640] (batch size 1, 3 color channels, 640x640 image)
image = Image.open("sunflower.jpg")
# Preprocess the image (resize, normalize, and convert to tensor)
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # Resize to 640x640
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Apply the transformations to the image
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension (1, 3, 640, 640)

# Trace the model with the image input
traced_model = torch.jit.trace(yolo_model, image_tensor)

# Save the traced model
traced_model.save('plant_leaf_detection_mobile.pt')

# Optionally, apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    traced_model, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8
)

# Save the quantized model
quantized_model.save('plant_leaf_detection_mobile_quantized.pt')

In [45]:
%pip install kivi

^C
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     ---------------------------------------- 0.5/317.3 MB 4.2 MB/s eta 0:01:15
     ---------------------------------------- 1.8/317.3 MB 5.0 MB/s eta 0:01:03
     ---------------------------------------- 2.9/317.3 MB 4.7 MB/s eta 0:01:08
     ---------------------------------------- 3.4/317.3 MB 4.4 MB/s eta 0:01:12
      --------------------------------------- 4.7/317.3 MB 4.6 MB/s eta 0:01:08
      --------------------------------------- 5.2/317.3 MB 4.6 MB/s eta 0:01:08
      --------------------------------------- 6.6/317.3 MB 4.5 MB/s eta 0:01:09
      --------------------------------------- 7.1/317.3 MB 4.4 MB/s eta 0:01:12
     - -------------------------------------- 8.1/317.3 MB 4.3 MB/s eta 0:01:12
     - -------------------------------------- 8.9/317.3 MB 4.3 MB/s eta 0:01:12
     - -------------------------------------- 9.4/317.

In [46]:
from PIL import Image as PILImage
import io,numpy as np
def preprocess_image(img_path):
    # Open the image using PIL and convert to RGB
    image = PILImage.open(img_path).convert('RGB')
    # Resize to the model's expected input size (640x640 for YOLO)
    image = image.resize((640, 640))
    # Convert to numpy array and normalize
    image_np = np.array(image).astype(np.float32)
    image_np /= 255.0  # Normalize to [0, 1]
    # Add batch dimension
    image_tensor = torch.from_numpy(image_np).permute(2, 0, 1).unsqueeze(0)  # [1, 3, 640, 640]
    return image_tensor

In [ ]:
mobile_model = torch.jit.load('plant_leaf_detection_mobile_quantized.pt')
mobile_model.eval()  # Set the model to evaluation mode

# Preprocess the image
image_tensor = preprocess_image("sunflower.jpg")

# Run inference
with torch.no_grad():
    output = mobile_model(image_tensor)  # Directly calling the model

# Print the prediction output
print("Prediction Output:", output)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()